<a href="https://colab.research.google.com/github/Rogerio-mack/fNIRS/blob/main/A_Carol_check_stimulus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verifica e Gera Lista de Arquivos somente com os Estímulos Completos

Isto é $ 3 \times 8  = 24 $ estímulos.

## Install & import `nilearn` and `mne-nirs`

In [1]:
#@markdown
!pip install nilearn
!pip install -U --no-deps https://github.com/mne-tools/mne-python/archive/main.zip
!pip install mne-nirs

from mne.io import read_raw_nirx
from mne.preprocessing.nirs import (optical_density, beer_lambert_law,
                                    temporal_derivative_distribution_repair)
import mne 



     - 61.5 MB 188 kB/s


## Mount Google Drive

In [2]:
#@markdown
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Common Imports

In [3]:
#@markdown 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
import seaborn as sns

import h5py
from itertools import compress
from sklearn.preprocessing import scale
from google.colab import widgets

## Routines

In [4]:
#@markdown `def clone_snirf(path_in, path_out, fname='out.snirf')`
def clone_snirf(path_in, path_out, fname='out.snirf'):
  
  if path_in == path_out:
    print('Error: path_in and path_out can not be the same.') 
    return

  if not os.path.exists(path_out):
      os.mkdir(path_out)
      print("Directory " , path_out ,  "was created")
  else:    
      print("Warning: directory " , path_out ,  " already exists")

  if os.path.exists(path_out + fname):
    os.remove(path_out + fname)
    print('Warning: previous output file was deleted.')
  
  print('Input snirf file: ' + path_in + fname)

  fs = h5py.File(path_in + fname,'r')
  fd = h5py.File(path_out + fname,'w')

  if list(fs.keys()).count('formatVersion') >= 1:
    fs.copy('formatVersion',fd,'formatVersion') 
  else:
    print('Warning: there is no formatVersion key in snirf input.')

  if list(fs.keys()).count('nirs') >= 1:
    fs.copy('nirs',fd,'nirs') 
  else:
    print('Error: Invalid snirf file. There is no nirs key in snirf input.')  
  
  print('Output snirf file: ' + path_out + fname)
  
  fd.close()
  fs.close()

  return

# Copia todos arquivos do drive para local

In [5]:
import os
path_in = '/content/drive/MyDrive/cond 1/snirf/'
# path_in = '/content/drive/MyDrive/Cond 2/snirf/'

path_out= '/content/'

In [6]:
# Clear all before copy... 
for f in os.listdir('/content/'):
  if '_run1' in f:
    os.system('rm ' + '/content/' + f )
  if '_run2' in f:
    os.system('rm ' + '/content/' + f )

In [7]:
entries = os.listdir(path_in)
print(entries)

for f in entries:
  if f.count('.snirf') > 0:
    os.system('cp ' + "'" + path_in + f  + "'" + ' ' + '/content/' + f )

['21_run1.snirf', '11_run1.snirf', '15_run1.snirf', '12_run1.snirf', '34_run1.snirf', '37_run1.snirf', '16_run1.snirf', '22_run1.snirf', '14_run1.snirf', '20_run1.snirf', '17_run1.snirf', '39_run1.snirf', '33_run1.snirf', '36_run1.snirf', '9_run1.snirf', '29_run1.snirf', '10_run1.snirf', '18_run1.snirf', '31_run1.snirf', '32_run1.snirf', '19_run1.snirf', '6_run1.snirf', '7_run1.snirf', '26_run1.snirf', '27_run1.snirf', '2_run1.snirf', '24_run1.snirf', '38_run1.snirf', '13_run1.snirf', '23_run1.snirf', '3_run1.snirf', '30_run1.snirf', '25_run1.snirf', '1_run1.snirf', '5_run1.snirf']


In [8]:
# for f in os.listdir(path_in):
#  if '_run1' in f:
#    os.system('rm ' + path_in + f )
if path_in == '/content/drive/MyDrive/Cond 2/snirf/':
  for f in os.listdir('/content/'):
    if '_run1' in f:
      os.system('rm ' + '/content/' + f )

if path_in == '/content/drive/MyDrive/cond 1/snirf/':
  for f in os.listdir('/content/'):
    if '_run2' in f:
      os.system('rm ' + '/content/' + f )

In [9]:
entries =  os.listdir(path_out)
entries = [x for x in entries if '.snirf' in x]

# entries

In [10]:
from mne.io import read_raw_nirx
from mne.preprocessing.nirs import (optical_density, beer_lambert_law,
                                    temporal_derivative_distribution_repair)
import mne
import pickle

threshold = 0.95

sci_df_exists = False
sci_df = pd.DataFrame()

raw_haemo_all = {}
epochs_all = {}

for fname in entries:
  raw_intensity = mne.io.read_raw_snirf(path_out + fname, verbose=True)
  raw_intensity.load_data()
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)

  raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od)

  raw_haemo = raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2,
                              l_trans_bandwidth=0.02)
  print(raw_haemo.filenames)

  events, event_dict = mne.events_from_annotations(raw_haemo)

  reject_criteria = dict(hbo=80e-6)
  tmin, tmax = -5, 15

  epochs = mne.Epochs(raw_haemo, events, event_id=event_dict,
                      tmin=tmin, tmax=tmax,
  #                    reject=reject_criteria, reject_by_annotation=True,
                      proj=True, baseline=(None, 0), preload=True,
                      detrend=None, verbose=True,
                      event_repeated='merge') # ADICIONADO Allowed values are 'error', 'drop', and 'merge'
  # epochs.plot_drop_log()

  raw_haemo_all[fname.split('.')[0]] = raw_haemo
  epochs_all[fname.split('.')[0]] = epochs


Loading /content/18_run1.snirf
Reading 0 ... 21573  =      0.000 ...  2157.300 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)



<ipython-input-10-4df4296734cb>:18: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)


('/content/18_run1.snirf',)
Used Annotations descriptions: ['Async1', 'Async2', 'Async3', 'Async4', 'Async5', 'Async6', 'Async7', 'Async8', 'Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 'controle1', 'controle2', 'controle3', 'controle4', 'controle5', 'controle6', 'controle7', 'controle8']
Not setting metadata
24 matching events found
Setting baseline interval to [-5.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 24 events and 201 original time points ...
0 bad epochs dropped
Loading /content/3_run1.snirf


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


Reading 0 ... 21603  =      0.000 ...  2160.300 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)



...

<ipython-input-10-4df4296734cb>:18: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


Setting baseline interval to [-5.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 24 events and 201 original time points ...
0 bad epochs dropped
Loading /content/21_run1.snirf
Reading 0 ... 21835  =      0.000 ...  2183.500 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)

('/content/21_run1.snirf',)
Used Annotations descriptions: ['Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 

<ipython-input-10-4df4296734cb>:18: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


Using data from preloaded Raw for 40 events and 201 original time points ...
0 bad epochs dropped
Loading /content/25_run1.snirf
Reading 0 ... 21724  =      0.000 ...  2172.400 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.05 - 0.7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.05
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.04 Hz)
- Upper passband edge: 0.70 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.80 Hz)
- Filter length: 1651 samples (165.100 sec)

('/content/25_run1.snirf',)
Used Annotations descriptions: ['Async1', 'Async2', 'Async3', 'Async4', 'Async5', 'Async6', 'Async7', 'Async8', 'Sync1', 'Sync2', 'Sync3', 'Sync4', 'Sync5', 'Sync6', 'Sync7', 'Sync8', 'controle1', 'controle2', 'controle3

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    0.1s finished


In [11]:
# raw_haemo_all 

In [12]:
# epochs_all

In [21]:
for f in entries: 
  if len(epochs_all[f.split('.')[0]].events) == 24:
    modelo = set(epochs_all[f.split('.')[0]].event_id.keys())
    last_samp = raw_haemo_all[f.split('.')[0]].last_samp/10


In [22]:
def check(subject,detail=False):
  
  if detail:
    print('número de eventos: ', len(epochs_all[subject].events))
    print()
    display(epochs_all[subject])
    print()
    print(epochs_all[subject].events)

    for i in range(len(raw_haemo_all[subject].annotations[:][:])):
      print(raw_haemo_all[subject].annotations[:][i]['description'].ljust(15),raw_haemo_all[subject].annotations[:][i]['onset'])

    print()

  for fname in [subject + '.snirf']: 
    path_out='/content/'

    data = h5py.File(path_out + fname,'r+')

    #
    # create alias for data['nirs']
    #
    for key in data['nirs'].keys():
      exp = key + ' = data["nirs"]["' + key + '"] '
    #  print(exp)
      exec(exp)

  if detail:
    for k in data['nirs'].keys():
      if k.count('stim') == 1:
        print( k, data['nirs'][k]['name'][:] , data['nirs'][k]['data'][:] )

    print()
  
  if len(epochs_all[subject].events) == 24:
    print(subject.ljust(15), 'OK')
    return subject, 'OK'

  diff = modelo - modelo.intersection(epochs_all[subject].event_id.keys())

  if len(epochs_all[subject].events) == 23 and len(diff) == 0:
    if raw_haemo_all[subject].last_samp/10 < data['nirs']['stim16']['data'][0][0] + tmax*10:
      print(subject.ljust(15), 'Tempo do Async8 ', data['nirs']['stim16']['data'][0][0], '+ tmax*10 deve ser menor que último sample ', last_samp)
      return subject, 'Tempo do Async8 + tmax*10 deve ser menor que último sample' 

  if len(diff) >= 1: 
    print(subject.ljust(15), 'Falta eventos: ', diff)
    return subject, 'Falta eventos: ' + str(diff) 

  print(subject.ljust(15), 'Com uma condição não prevista')
  return subject, 'Com uma condição não prevista'
  



In [27]:
subject = []
comment = []

for f in entries: 
  s , c = check(f.split('.')[0])
  subject.append(s)
  comment.append(c)

check_stim = pd.DataFrame()
check_stim['subject'] = subject
check_stim['comment'] = comment

check_stim_view = check_stim.style.set_properties(**{'text-align': 'left'})


18_run1         OK
3_run1          OK
11_run1         Falta eventos:  {'Async6', 'Async5', 'Async7', 'Async3', 'Async4', 'Async8', 'Async2'}
20_run1         OK
9_run1          OK
24_run1         OK
12_run1         OK
39_run1         OK
17_run1         OK
37_run1         OK
30_run1         OK
36_run1         OK
33_run1         OK
1_run1          Falta eventos:  {'controle3', 'Sync3', 'controle7', 'controle1', 'controle5', 'Async4', 'controle4', 'Async2', 'Sync8', 'Sync2', 'Async1', 'Async6', 'Sync4', 'Sync7', 'Async7', 'Async5', 'Sync6', 'controle6', 'controle8', 'Async3', 'Async8', 'controle2', 'Sync5', 'Sync1'}
13_run1         OK
23_run1         OK
27_run1         OK
14_run1         OK
34_run1         OK
16_run1         OK
29_run1         OK
15_run1         OK
2_run1          OK
38_run1         OK
26_run1         OK
5_run1          OK
10_run1         OK
31_run1         OK
6_run1          OK
7_run1          Tempo do Async8  2164.0 + tmax*10 deve ser menor que último sample  2172.4
32_r

In [30]:
display(check_stim_view)

,subject,comment
0,18_run1,OK
1,3_run1,OK
2,11_run1,"Falta eventos: {'Async6', 'Async5', 'Async7', 'Async3', 'Async4', 'Async8', 'Async2'}"
3,20_run1,OK
4,9_run1,OK
5,24_run1,OK
6,12_run1,OK
7,39_run1,OK
8,17_run1,OK
9,37_run1,OK


In [32]:
if path_in == '/content/drive/MyDrive/Cond 2/snirf/':
  check_stim.to_csv('check_stim_cond_2.csv',index=None)

if path_in == '/content/drive/MyDrive/cond 1/snirf/':
  check_stim.to_csv('check_stim_cond_1.csv',index=None)